# Set up

In [ ]:
using PyCall
dj = pyimport("datajoint")

# RUN THE FOLLOWING ONLY ONCE:  Next time tou start up tou won't need it, the info will be saved in the local file
#
# dj.config.__setitem__("database.host", "datajoint00.pni.princeton.edu")
# dj.config.__setitem__("database.user", "YOUR PU ID")
# dj.config.__setitem__("database.password", "YOUR PU PASSWORD")
# dj.config.save_local()

In [2]:
# And now connect:

dj.conn()

PyObject DataJoint connection (connected) brody@datajoint00.pni.princeton.edu:3306

# Making a schema

In [ ]:
schema = dj.schema("brody_tutorial2", py"locals()")


# Defining a table

The table is defined as a Python class, so we go into Python through PyCall's `py"""` syntax and declare it there.
The `$` signs tell PyCall to use the Julia variable at that point, but within the Python environment (e.g., `schema` and `dj`).

In [ ]:
py"""
@$schema
class Mouse($dj.Manual):
      definition = '''
      mouse_id: int                  # unique mouse id
      ---
      dob: date                      # mouse date of birth
      sex: enum('M', 'F', 'U')       # sex of mouse - Male, Female, or Unknown/Unclassified
      '''
"""

Having defined the `Mouse` class in Python, now we instantiate an instance of it. (The class was defined in the Python environment, not Julia; so we use PyCall's inline `py"` syntax here to instantiate in the Python environment). We'll then store the result in a Julia variable, in this case `mouse`.

In [3]:
mouse = py"Mouse()"

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
1,2019-12-19,M
2,2019-12-20,U
3,2019-12-21,M


┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = show(::IOContext{Base.GenericIOBuffer{Array{UInt8,1}}}, ::MIME{Symbol("text/html")}, ::PyObject) at PyCall.jl:889
└ @ PyCall /Users/carlos/.julia/packages/PyCall/ttONZ/src/PyCall.jl:889


# Inserting data into a table

works just like in Python.  For example, we can execute the code below. (Note that if the table and these primary keys already existed, duplicate entry errors will be generated. To avoid those errors, and silently skip inserting duplicates, use the `skip_duplicates=true flag`)


In [7]:
mouse.insert1([1, "2019-12-19", "M"])   # A single row, with columns following the order in the table definition
mouse.insert1(Dict("mouse_id"=>2, "dob"=>"2019-12-20", "sex"=>"U"))  # Using a dictionary

In [8]:
mouse.insert1([1, "2019-12-19", "M"], skip_duplicates=true)   # A single row, with columns following the order in the table definition

And using `mouse.insert()` (no "1" at the end of `insert`) you can send a list of tuples or a list of dictionaries to insert multiple rows:

In [33]:
mouse.insert([
        Dict("mouse_id"=>5, "dob"=>"2020-01-05", "sex"=>"M"), 
        Dict("mouse_id"=>6, "dob"=>"2020-01-05", "sex"=>"F"),
        Dict("mouse_id"=>7, "dob"=>"2020-01-05", "sex"=>"F")
        ], skip_duplicates=true)

In [12]:
mouse

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
1,2019-12-19,M
2,2019-12-20,U
3,2019-12-21,M
4,2020-01-03,F
5,2020-01-05,M
6,2020-01-05,F
7,2020-01-05,F


You can now fetch data, and do restrictions, just like in Python, but note that we use single quotes for non-numeric things inside the double quotes, so the two types of quote symbol don't mix up with each other.

In [17]:
mouse & "dob > '2020-01-03'"

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
5,2020-01-05,M
6,2020-01-05,F
7,2020-01-05,F


In [20]:
mouse & "dob = '2020-01-05'" & "sex = 'F'"

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
6,2020-01-05,F
7,2020-01-05,F


In [24]:
mouse & Dict("sex"=>"M")

mouse_id unique mouse id,dob mouse date of birth,"sex sex of mouse - Male, Female, or Unknown/Unclassified"
1,2019-12-19,M
3,2019-12-21,M
5,2020-01-05,M


# Fetching data

Any table object (without restrictions or after a restriction) can have data fetched from it. Note that the data comes back as a PyObject; below we'll convert that into Julia.

In [21]:
mouse.fetch()

PyObject array([(1, datetime.date(2019, 12, 19), 'M'),
       (2, datetime.date(2019, 12, 20), 'U'),
       (3, datetime.date(2019, 12, 21), 'M'),
       (4, datetime.date(2020, 1, 3), 'F'),
       (5, datetime.date(2020, 1, 5), 'M'),
       (6, datetime.date(2020, 1, 5), 'F'),
       (7, datetime.date(2020, 1, 5), 'F')],
      dtype=[('mouse_id', '<i8'), ('dob', 'O'), ('sex', 'O')])

We'll define a function to take the output of the Python function `fetch()` and turn it into Julia format:

In [34]:
import Dates

# List of data types that are not automatically converted, defining how to convert
# First column is Python name for the data type class, second column is corresponding Julia type
conversion_list = [
      "str"   String;
      "date"  Dates.Date;
]

"""
function d2j(x)

Takes x, the output of a datajoint fetch() call, and converts it from PyObject formal to Julia format

= PARAMETERS:

- x      Must be a PyObject that is the output of a datajoint fetch() call

= RETURNS:

- out    The same data that was in x, but now in Julia format. In general this will
         be an Array of type Any, but if x was a single column of unique type, or a list
         of single columns, each of unique type, then those may come back as the corresponding
         type (e.g., Array of Int or Array of String).

= EXAMPLE CALL:

```jldoctest
d2j(mouse.fetch())

7×3 Array{Any,2}:
 1  2019-12-19  "M"
 2  2019-12-20  "U"
 3  2019-12-21  "M"
 4  2020-01-03  "F"
 5  2020-01-05  "M"
 6  2020-01-05  "F"
 7  2020-01-05  "F"

d2j(mouse.fetch("mouse_id"))

7-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
 7


```
"""
function d2j(x)
      out = x
      if typeof(x) <: PyObject && x.__class__.__name__ == "ndarray" &&  !isempty(x)
            # We are getting each row of the table as a row of x.
            # This means different columns will have different data types
            nrows = length(x)
            ncols = length(get(x, 0))
            out = Array{Any}(undef, nrows, ncols)
            for i=1:nrows
                  for j=1:ncols
                        # These get statements do autoconvert to Julia data types
                        out[i,j] = get(get(x, i-1), j-1)
                        # Except for integers it seems, which stay as PyObjects but can be
                        # converted as follows:
                        if typeof(out[i,j]) <: PyObject && py"hasattr($(out[i,j]), 'flatten')"
                              out[i,j] = out[i,j].flatten()[1]
                        end
                  end
            end
            return out
      elseif typeof(x) <: Array{PyObject,1}
            # We have a single column requested: ony a single data type
            if !isempty(x)
                  classname = x[1].__class__.__name__
                  # Do we know how to convert this data type?
                  u = findall(conversion_list[:,1] .== classname)
                  if !isempty(u)
                        # We do know!
                        u = u[1]
                        mytype = conversion_list[u,2]
                        out = Array{mytype}(undef, length(x))
                        for i=1:length(x)
                              out[i] = convert(mytype, x[i])
                        end
                  end
            end
      elseif typeof(x) <: Array && eltype(x) <: Array
            # We have multiple columns requested as multiple outputs
            # We'll convert each one separately
            out = Array{Any}(undef, length(x))
            for i=1:length(x)
                  out[i] = d2j(x[i])
            end
      end
      return out
end


d2j

In [35]:
d2j(mouse.fetch())

7×3 Array{Any,2}:
 1  2019-12-19  "M"
 2  2019-12-20  "U"
 3  2019-12-21  "M"
 4  2020-01-03  "F"
 5  2020-01-05  "M"
 6  2020-01-05  "F"
 7  2020-01-05  "F"

In [36]:
d2j(mouse.fetch("mouse_id"))

7-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
 7

In [37]:
mouse_id, sex = d2j(mouse.fetch("mouse_id", "sex"));
println("mouse id is"); println(mouse_id)
println("sex is");      println(sex)


mouse id is
[1, 2, 3, 4, 5, 6, 7]
sex is
["M", "U", "M", "F", "M", "F", "F"]


In [ ]:
acquisition = dj.create_virtual_module("acquisition", "u19_acquisition")

In [ ]:
py"$dj.Diagram($acquisition)"